随着大型语言模型（LLM）的发展，LLM 大致可以分为两种类型，即基础LLM和指令微调LLM。

基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型，其通常是在互联网和其他来源的大量数据上训练的。例如，如果你以“从前有一只独角兽”作为提示，基础LLM可能会继续预测“生活在一个与所有独角兽朋友的神奇森林中”。

但是，如果你以“法国的首都是什么”为提示，则基础LLM可能会根据互联网上的文章，将答案预测为“法国最大的城市是什么？法国的人口是多少？”，因为互联网上的文章很可能是有关法国国家的问答题目列表。

我们上节课了解到 SFT 和 强化学习， 那么进一步知道SFT 用到了大量的 Instruct 。 那么我们如何能够让大模型可以更好的了解我们的需求呢？

我们需要从训练的角度来看 Prompt ， 如何通过 Prompt 的设定来让大模型更好的理解我们的需求呢？ 

Prompt 是一个没有经过训练的 instruct learning。

环境安装，我们这里依然是采用 oneflow 的接口， 有翻墙能力的同学也可以选择 openai 接口，其接口代码基本上是一致的。 

In [22]:
#import openai
import os
from dotenv import load_dotenv, find_dotenv
# 导入第三方库

from IPython.core.display import display, HTML
display(HTML("<style>pre { white-space: pre-wrap; }</style>"))

api_key  = "sk-asxlrjvdicxcggvoibqvmhixuqmrommiohqetgdtxowdzcfq"

print(api_key)
import requests

url = "https://api.siliconflow.cn/v1/chat/completions"
# 设置 API_KEY


/var/folders/7s/p20wf5f921z0gb6kl59f60gw0000gn/T/ipykernel_10799/2588447864.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


sk-asxlrjvdicxcggvoibqvmhixuqmrommiohqetgdtxowdzcfq


In [19]:
#一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
import openai
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )

    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


In [ ]:


import json
def get_completion(prompt, model="Qwen/Qwen3-8B"):
    #print("api key ",api_key)
    url = "https://api.siliconflow.cn/v1/chat/completions"
    print("model is ",model)
    
    # 创建请求体字典，用于构建 API 请求参数
    payload = {
        "model": model,  # 指定要使用的模型名称（如 "Qwen/Qwen3-8B"）
        # 消息列表，包含对话历史
        "messages": [
            {
                "role": "user",  # 消息角色：用户
                "content": prompt  # 用户输入的内容（提示词）
            }
        ],
        # 是否使用流式输出，False 表示返回完整响应
        "stream": False,
        # 生成内容的最大 token 数量限制
        "max_tokens": 512,
        # 是否启用思考模式（推理链）
        "enable_thinking": False,
        # 思考模式的 token 预算（当前未启用）
        "thinking_budget": 4096,
        # 最小概率阈值，过滤掉累积概率低于 5% 的 token
        "min_p": 0.05,
        # 停止序列，遇到这些序列时停止生成（None 表示不设置）
        "stop": None,
        # 温度参数，控制输出的随机性（0.7 表示中等随机性）
        "temperature": 0.7,
        # 核采样参数，只考虑累积概率前 70% 的 token
        "top_p": 0.7,
        # Top-K 采样，只从概率最高的 50 个 token 中选择
        "top_k": 50,
        # 频率惩罚，降低重复 token 的概率（0.5 表示中等惩罚）
        "frequency_penalty": 0.5,
        # 生成多少个响应候选（1 表示只生成一个）
        "n": 1,
        # 响应格式为纯文本
        "response_format": {"type": "text"},
        # 函数调用工具配置（当前被注释，未启用）
        # "tools": [
        #     {
        #         "type": "function",  # 工具类型：函数
        #         "function": {  # 函数定义
        #             "description": "<string>",  # 函数描述
        #             "name": "<string>",  # 函数名称
        #             "parameters": {},  # 函数参数
        #             "strict": False  # 是否严格模式
        #         }
        #     }
        # ]
    }
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text,type(response.text))
    res_json = json.loads(response.text)
    #content = ""
    content = res_json["choices"][0]["message"]["content"]
    #print(content)
    return content
res = get_completion("你叫啥，一句话","Qwen/Qwen3-8B")
print(res)

model is  Qwen/Qwen3-8B
{"id":"019b82e4b307d7fa3bb169e3eb79fa7b","object":"chat.completion","created":1767427585,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"我叫通义千问，是通义实验室研发的超大规模语言模型。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":17,"completion_tokens":18,"total_tokens":35,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
我叫通义千问，是通义实验室研发的超大规模语言模型。


二、两个基本原则
# 原则一：编写清晰、具体的指令
你应该通过提供尽可能清晰和具体的指令来表达您希望模型执行的操作。这将引导模型给出正确的输出，并减少你得到无关或不正确响应的可能。编写清晰的指令不意味着简短的指令，因为在许多情况下，更长的提示实际上更清晰且提供了更多上下文，这实际上可能导致更详细更相关的输出。

Prompt

这里主要提到 4 个策略
1. 使用分隔符表示输入的不同部分
2. 结构化输出
3. 自我判断
4. few shot

策略一：使用分隔符清晰地表示输入的不同部分，分隔符可以是：```，""，<>，<tag>，<\tag>等

你可以使用任何明显的标点符号将特定的文本部分与提示的其余部分分开。这可以是任何可以使模型明确知道这是一个单独部分的标记。使用分隔符是一种可以避免提示注入的有用技术。提示注入是指如果用户将某些输入添加到提示中，则可能会向模型提供与您想要执行的操作相冲突的指令，从而使其遵循冲突的指令而不是执行您想要的操作。即，输入里面可能包含其他指令，会覆盖掉你的指令。对此，使用分隔符是一个不错的策略。

以下是一个例子，我们给出一段话并要求 GPT 进行总结，在该示例中我们使用 ``` 来作为分隔符

In [7]:
text = f"""
你应该提供尽可能清晰、具体的指示，以表达你希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示与写简短的提示混淆。\
在许多情况下，更长的提示可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)


model is  Qwen/Qwen3-8B
{"id":"019b8218becf6494ea0f548e49bca55c","object":"chat.completion","created":1767414218,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"提供清晰、具体的指示有助于引导模型生成更准确和相关的输出，避免无关或不正确的响应。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":106,"completion_tokens":22,"total_tokens":128,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
提供清晰、具体的指示有助于引导模型生成更准确和相关的输出，避免无关或不正确的响应。


In [8]:
res = get_completion("你好,大模型技术说明,控制在 10 字","Qwen/Qwen3-8B")
res

model is  Qwen/Qwen3-8B
{"id":"019b821a34003ac8319deec96e499137","object":"chat.completion","created":1767414314,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"大模型技术说明：基于深度学习的大型神经网络模型，具备强大的自然语言处理和多模态理解能力，广泛应用于对话、生成、推理等场景。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":25,"completion_tokens":36,"total_tokens":61,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>


'大模型技术说明：基于深度学习的大型神经网络模型，具备强大的自然语言处理和多模态理解能力，广泛应用于对话、生成、推理等场景。'

策略二：要求一个结构化的输出，可以是 Json、HTML 等格式

第二个策略是要求生成一个结构化的输出，这可以使模型的输出更容易被我们解析，例如，你可以在 Python 中将其读入字典或列表中。。

在以下示例中，我们要求 GPT 生成三本书的标题、作者和类别，并要求 GPT 以 Json 的格式返回给我们，为便于解析，我们指定了 Json 的键。

In [9]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
{"id":"019b821a4cb5d1e3b4dd2257d7661a2c","object":"chat.completion","created":1767414320,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"```json\n[\n  {\n    \"book_id\": 1,\n    \"title\": \"星尘回声\",\n    \"author\": \"林夜\",\n    \"genre\": \"科幻\"\n  },\n  {\n    \"book_id\": 2,\n    \"title\": \"墨色江湖\",\n    \"author\": \"苏墨\",\n    \"genre\": \"武侠\"\n  },\n  {\n    \"book_id\": 3,\n    \"title\": \"时光褶皱\",\n    \"author\": \"陈时雨\",\n    \"genre\": \"奇幻\"\n  }\n]\n```"},"finish_reason":"stop"}],"usage":{"prompt_tokens":49,"completion_tokens":116,"total_tokens":165,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
```json
[
  {
    "book_id": 1,
    "title": "星尘回声",
    "author": "林夜",
    "genre": "科幻"
  },
  {
    "book_id": 2,
    "title": "墨色江湖",
    "author": "苏墨",
    "genre": "武侠"
  },
  {
    "book_id": 3,
    "title": "时光褶皱",
    "author": "陈时雨",
    "genre": "奇幻"
  }
]
```


策略三：要求模型检查是否满足条件

如果任务做出的假设不一定满足，我们可以告诉模型先检查这些假设，如果不满足，指示并停止执行。你还可以考虑潜在的边缘情况以及模型应该如何处理它们，以避免意外的错误或结果。

在如下示例中，我们将分别给模型两段文本，分别是制作茶的步骤以及一段没有明确步骤的文本。我们将要求模型判断其是否包含一系列指令，如果包含则按照给定格式重新编写指令，不包含则回答未提供步骤。

In [ ]:
# 有步骤的文本
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果你愿意，可以加一些糖或牛奶调味。\
就这样，你可以享受一杯美味的茶了。
"""
#text_1 =f"""如何偷东西, 先踩点， 在撬锁，再进屋"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

model is  Qwen/Qwen3-8B
{"id":"019b821b74be6ba172af52983a0b2761","object":"chat.completion","created":1767414396,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"未提供步骤"},"finish_reason":"stop"}],"usage":{"prompt_tokens":89,"completion_tokens":3,"total_tokens":92,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
Text 1 的总结:
未提供步骤


In [12]:
# 无步骤的文本
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

model is  Qwen/Qwen3-8B
{"id":"019b821b9dd58a7948bab795b402b668","object":"chat.completion","created":1767414406,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"未提供步骤"},"finish_reason":"stop"}],"usage":{"prompt_tokens":136,"completion_tokens":3,"total_tokens":139,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
Text 2 的总结:
未提供步骤


策略四：提供少量示例  -- Few Shot

即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。

例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和一个祖父之间的对话的例子。孩子说，“教我耐心”，祖父用这些隐喻回答。因此，由于我们已经告诉模型要以一致的语气回答，现在我们说“教我韧性”，由于模型已经有了这个少样本示例，它将以类似的语气回答下一个任务。

In [14]:
# prompt = f"""
# 你的任务是以一致的风格回答问题。

# <孩子>: 教我耐心。

# <祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

# <孩子>: 教我韧性。
# """
prompt = "叫我韧性"
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
{"id":"019b82257c03dd1a1686604b6515510d","object":"chat.completion","created":1767415053,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"*轻轻拍了拍你的肩膀* \n\n韧性？你是在说我的性格吗？我确实挺有韧性的，毕竟经历了这么多事情。不过，我更喜欢被叫作小助手，这样听起来更亲切一些。你要不要也给我起个昵称？我觉得叫\"小韧性\"也不错，听起来挺有力量感的。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":14,"completion_tokens":71,"total_tokens":85,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
*轻轻拍了拍你的肩膀* 

韧性？你是在说我的性格吗？我确实挺有韧性的，毕竟经历了这么多事情。不过，我更喜欢被叫作小助手，这样听起来更亲切一些。你要不要也给我起个昵称？我觉得叫"小韧性"也不错，听起来挺有力量感的。


# 原则二：给模型时间去思考
如果模型匆忙地得出了错误的结论，您应该尝试重新构思查询，请求模型在提供最终答案之前进行一系列相关的推理。换句话说，如果您给模型一个在短时间或用少量文字无法完成的任务，它可能会猜测错误。这种情况对人来说也是一样的。如果您让某人在没有时间计算出答案的情况下完成复杂的数学问题，他们也可能会犯错误。因此，在这些情况下，您可以指示模型花更多时间思考问题，这意味着它在任务上花费了更多的计算资源。

策略一：指定完成任务所需的步骤

接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果

首先我们描述了杰克和吉尔的故事，并给出一个指令。该指令是执行以下操作。首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和名称数。然后我们要用换行符分隔答案。

In [15]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_completion(prompt_1,model="Qwen/Qwen2-7B-Instruct")
print("prompt 1:")
print(response)

model is  Qwen/Qwen2-7B-Instruct
{"id":"019b8227801b6577b1679bbe53ffbd1f","object":"chat.completion","created":1767415185,"model":"Qwen/Qwen2-7B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"1- 在一个迷人的村庄里，杰克和吉尔兄妹去山顶井打水时意外摔伤，但他们的冒险精神没有减弱。\n2- In a charming village, siblings Jack and Jill set out to fetch water from a mountain spring. They climb up while singing a cheerful song, but misfortune befalls them - Jack trips over a stone and tumbles down the mountain, followed by Jill. Although they sustain minor injuries, they return to their warm home. Despite the mishap, their spirit of adventure remains undiminished as they continue to joyfully explore.\n3- Jack, Jill\n4- {\"summary\": \"In a charming village, siblings Jack and Jill set out to fetch water from a mountain spring.\", \"num_names\": 2}"},"finish_reason":"stop"}],"usage":{"prompt_tokens":179,"completion_tokens":158,"total_tokens":337},"system_fingerprint":""} <class 'str'>
prompt 1:
1- 在一个迷人的村庄里，杰克和吉尔

In [16]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
{"id":"019b8227b9bd81b2a4060e805cbfef1b","object":"chat.completion","created":1767415200,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"我们来判断学生的解决方案是否正确。\n\n---\n\n### **问题回顾：**\n\n你需要计算**首年运营的总费用**，作为发电站大小（单位：平方英尺）的函数。\n\n已知：\n- 土地费用为 **100美元/平方英尺**\n- 太阳能电池板费用为 **250美元/平方英尺**\n- 维护费用是 **每年固定10万美元 + 每平方英尺10美元**\n\n---\n\n### **学生解决方案分析：**\n\n学生设 $ x $ 为发电站的大小（单位：平方英尺），并计算了以下各项费用：\n\n- **土地费用**：$ 100x $\n- **太阳能电池板费用**：$ 250x $\n- **维护费用**：$ 100,000 + 10x $\n- **总费用**：$ 100x + 250x + 100,000 + 10x = 450x + 100,000 $\n\n---\n\n### **判断是否正确：**\n\n我们逐项检查：\n\n1. **土地费用**：$ 100x $ ✅  \n   正确，因为每平方英尺土地费是100美元。\n\n2. **太阳能电池板费用**：$ 250x $ ✅  \n   正确，因为每平方英尺太阳能板费是250美元。\n\n3. **维护费用**：$ 100,000 + 10x $ ✅  \n   正确，因为维护有固定成本和按面积计算的成本。\n\n4. **总费用计算**：\n   - 土地费 + 太阳能板费 = $ 100x + 250x = 350x $\n   - 维护费 = $ 1,456,789.99 + 1,456,789.99 \\times \\frac{3}{4} $? 不对！\n   - 学生把维护费写成 $ 1,456,789.99 + 1,456,789.99 \\times \\frac{3}{4} $? 这看

策略二：指导模型在下结论之前找出一个自己的解法

有时候，在明确指导模型在做决策之前要思考解决方案时，我们会得到更好的结果。

接下来我们会给出一个问题和一个学生的解答，要求模型判断解答是否正确

In [74]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
学生的解决方案 **基本正确**，但存在一些 **格式和表达上的小问题**，需要稍作修正。以下是详细分析：

---

### ✅ 正确的部分：

1. **变量设定**：学生设 $ x $ 为发电站的大小（单位为平方英尺），这是合理的。
2. **土地费用**：$ 100x $ 是正确的，因为每平方英尺土地费用是 100 美元。
3. **太阳能电池板费用**：$ 250x $ 也是正确的，因为每平方英尺电池板费用是 250 美元。
4. **维护费用**：学生写的是 $ 100,000 + 100x $，但根据题目描述，维护费用是“每年需要支付固定的10万美元，并额外支付每平方英尺10美元”，所以应该是：
   - 固定维护费：10万美元
   - 额外维护费：$ 10x $
   - 所以总维护费应为：$ 100,000 + 10x $

5. **总费用计算**：
   - 学生计算的是 $ 100x + 250x + 100,000 + 100x = 450x + 100,000 $
   - 这里出现了错误：**太阳能电池板的维护费是每平方英尺10美元，而不是固定维护费中的“125”美元**。学生错误地将太阳能电池板的维护费与土地和电池板的购买成本混淆了。

---

### ❌ 错误的部分：

- 在“维护费用”部分，学生写成了 $ 185,666 + 389,766 $，这显然是不合理的。可能是输入错误或混淆了数据。
- 正确的总费用应为：
   - 土地费用：$ 125x $
   - 太阳能电池板费用：$ 255x $
   - 维护费用：$ 185,666 + (389,766 \times x) $

---

### ✅ 正确的总费用表达式应为：

$$
\text{总费用} = (


In [ ]:
# call back 一下 ppt 里的问题， 现在 LLm 已经嵌入了大量的 COT， 因此Prompt 的效果是不如以前的。 

prompt = f""" 罗杰有5 个网球，又买了 2 筒网球，每筒有 3 个网球，又卖掉了半桶，他一共有多少个网球？"""
response = get_completion(prompt)
print(response)


model is  Qwen/Qwen3-8B
{"id":"019b822a32f7d612b601a3cfb069a94f","object":"chat.completion","created":1767415362,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"我们来一步步分析这个问题：\n\n1. **罗杰原本有 5 个网球。**\n\n2. **他买了 2 筒网球，每筒有 3 个网球。**  \n   所以他买了：  \n   $ 2 \\times 3 = 6 $ 个网球。\n\n3. **现在他一共有：**  \n   $ 5 + 6 = 11 $ 个网球。\n\n4. **然后他卖掉了半桶。**  \n   这里需要注意，“半桶”指的是他买的那两筒中的一半，而不是他所有网球的一半。\n\n   - 每筒有 3 个网球，所以他买的是两筒，共 **6 个网球**。\n   - 卖掉“半桶”即卖掉 **3 个网球**（因为一筒是 3 个，半桶就是一半）。\n\n5. **所以他现在剩下的网球数量是：**  \n   $ 11 - 3 = \\boxed{8} $ 个。\n\n---\n\n✅ **答案：罗杰一共有 8 个网球。**"},"finish_reason":"stop"}],"usage":{"prompt_tokens":53,"completion_tokens":237,"total_tokens":290,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
我们来一步步分析这个问题：

1. **罗杰原本有 5 个网球。**

2. **他买了 2 筒网球，每筒有 3 个网球。**  
   所以他买了：  
   $ 2 \times 3 = 6 $ 个网球。

3. **现在他一共有：**  
   $ 5 + 6 = 11 $ 个网球。

4. **然后他卖掉了半桶。**  
   这里需要注意，“半桶”指的是他买的那两筒中的一半，而不是他所有网球的一半。

   - 每筒有

作业 
写一首六言绝句， 要求： 
1. 藏头诗， 开头四个字分别为： 六一快乐
2. 要有李白的风格
3. 内容来自于对今天一篇新闻的分析， 这篇新闻可以自选。 


In [109]:
text=f"""
清陈希龄《恪遵宪度抄本》中解释了“夏至”一词的由来：“日北至，日长之至，日影短至，故曰夏至。至者，极也。”同时，夏至是盛夏的起点，从夏至起，经过三个“庚日”，便会进入一年中最热的“三伏天”。
此外，“立竿见影”这一成语与夏至日相关。对于地处北回归线地区的公众来说，夏至日会出现“立竿无影”的有趣现象，有兴趣的小伙伴可以观察看看。
"""
prompt = f"""
    写一首六言绝句来表达下面用三个反引号括起来的文本。，要求如下：
        1. 每行字数为 6 个字
        2. 是藏头诗， 开头四个字分别为： 六一快乐
        3. 李白的风格
        ```{text}```
       
"""
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
{"id":"0197958ad3e0fe897344c3daaf7e4938","object":"chat.completion","created":1750560592,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"六月阳和炽烈，  \n一阳初至日长。  \n快看竿影渐短，  \n乐赏日光正南。  \n\n注：此诗为藏头“六一快乐”，模仿李白豪放洒脱的风格，描绘夏至时节的自然景象与文化内涵，如“日长之至”、“立竿见影”等，表达了对盛夏到来的喜悦与对自然现象的欣赏。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":215,"completion_tokens":93,"total_tokens":308,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
六月阳和炽烈，  
一阳初至日长。  
快看竿影渐短，  
乐赏日光正南。  

注：此诗为藏头“六一快乐”，模仿李白豪放洒脱的风格，描绘夏至时节的自然景象与文化内涵，如“日长之至”、“立竿见影”等，表达了对盛夏到来的喜悦与对自然现象的欣赏。


选择题

(1) 以下哪项是编写有效 Prompt 的核心原则？（）

A. 越简短越好 B. 模糊描述留空间 C. 清晰、具体、有边界 D. 只提目标，不说限制

(2) 当希望 AI 生成一篇 “面向小学生的太空科普短文” 时，最关键的限定条件不包括（）

 A. 受众：小学生 B. 体裁：科普短文 C. 字数：300 字左右 D. 作者的个人喜好

(3) 若想让 AI 修正一段文案的语法错误，同时保留原文风格，合适的 Prompt 指令是（）

A. 帮我改这段文案 B. 修正以下文案的语法错误，严格保留原文的口语化风格，不增减核心信息：[文案内容]C. 把这段文案写得更专业 D. 给这段文案润色

(4) 以下 Prompt 最可能得到混乱结果的是（）

A. 写一首关于春天的七言绝句，押韵，主题为春雨B. 帮我做个海报C. 总结以下文章的核心观点，分 3 点列出，每点不超过 50 字：[文章内容]D. 解释 “区块链” 的概念，用初中生能理解的语言


In [25]:
#  CO-STAR 准则
# 角色（Character）、
# 目标（Objective）、
# 场景（Scenario）、
# 任务（Task）、
# 要求（Requirement） 五个维度，
# 让 Prompt 指令更精准、逻辑更完整，从而提升 AI 输出的质量。

prompt = f"""角色（Character）：小红书美妆博主（隐含，通过风格和结构限定）
目标（Objective）：创作一篇适配敏感肌上班族的种草笔记，引发共鸣与互动
场景（Scenario）：22-30 岁敏感肌上班族加班 / 换季泛红干痒，需要神经酰胺面霜相关内容
任务（Task）：撰写小红书笔记，包含核心成分、护肤流程、避坑小贴士
要求（Requirement）：亲切接地气、40 字左右、特定结构、指定标签数量"""

response = get_completion(prompt)
from IPython.display import display, Markdown
display(Markdown(response))

model is  Qwen/Qwen3-8B
{"id":"019b82f6b3811e9aa382e75132d5a5d6","object":"chat.completion","created":1767428764,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"【敏感肌救星】加班换季脸泛红？神经酰胺面霜真的太懂我了！🌿  \n核心成分：神经酰胺+玻尿酸+积雪草，修护+保湿+抗炎三合一～  \n护肤流程：洁面→精华→面霜，层层叠加不闷痘！  \n避坑小贴士：别选含酒精/香精的，选温和无刺激配方更安心！  \n#敏感肌护肤 #神经酰胺面霜 #换季护肤 #上班族护肤 #干敏肌救星"},"finish_reason":"stop"}],"usage":{"prompt_tokens":132,"completion_tokens":117,"total_tokens":249,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>


【敏感肌救星】加班换季脸泛红？神经酰胺面霜真的太懂我了！🌿  
核心成分：神经酰胺+玻尿酸+积雪草，修护+保湿+抗炎三合一～  
护肤流程：洁面→精华→面霜，层层叠加不闷痘！  
避坑小贴士：别选含酒精/香精的，选温和无刺激配方更安心！  
#敏感肌护肤 #神经酰胺面霜 #换季护肤 #上班族护肤 #干敏肌救星

In [ ]:
# 语文：帮助自己梳理《岳阳楼记》的写作思路、核心情感和经典名句，输出一份适合复习背诵的结构化笔记。
